In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, SGD
from keras.backend.tensorflow_backend import set_session

import tensorflow as tf

from utils.custom_loss import mae_wrap_angle, mse_wrap_angle
from utils.load_data_raw import DataGenerator_raw, load_raw_ft_h5, load_raw_IDs_h5
from utils.dataset_split import create_split

In [ ]:
# configure tf session
config = tf.ConfigProto(device_count={'CPU': 2,
                                      'GPU': 0},
                        intra_op_parallelism_threads=6, # num of cores per socket
                        inter_op_parallelism_threads=2,
                        allow_soft_placement=True)
sess = tf.Session(config=config)
set_session(sess)

# config as suggested here:
# https://software.intel.com/en-us/articles/tips-to-improve-performance-for-popular-deep-learning-frameworks-on-multi-core-cpus

## Model

In [ ]:
def simple_model(in_dim):
    """Define simple model"""
    # create model
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(in_dim,)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    # show info
    model.summary()
    # compile model
    model.compile(loss = mae_wrap_angle,
                  optimizer = 'adam',
                  metrics = [mse_wrap_angle])
    return model

## Training

In [ ]:
# define training parameters
batch_size = 1024
num_epochs = 200

# define filename of file containing dataset
filename = '/media/feliximmohr/Storage/master_thesis/generated/database/raw/raw_nf10_mid/database_raw_nf10_scaledMM.h5'

# list of substrings of parameters to select, e.g. 'NFCHOA', 'pos10', 'R006'
valid_subset = [] #['LWFS','R006','M006','M027','R027','M013','R013']
test_subset = ['NFCHOA_R006']

In [ ]:
# load data indices and reference tables
ID_ref, pos_table, cond_table, par = load_raw_IDs_h5(filename)
# create test/validation/train dataset split
partition = create_split(ID_ref,cond_table,test_subset,valid_split=0.2)
# load feature and target data
feature_data, target_data, _ = load_raw_ft_h5(filename)

In [ ]:
# model.fit_generator on batches of dataset

# define generators
params = {'dim': feature_data.shape[1],
          'batch_size': batch_size,
          'feature_data': feature_data.values,
          'target_data' : target_data.values,
          'shuffle': True,
          'n_frames': par['nFrames']
         }
train_batch_generator = DataGenerator_raw(partition['train'], **params)
valid_batch_generator = DataGenerator_raw(partition['validation'], **params)

# create model
model = simple_model(feature_data.shape[1])

# define callbacks
csv_logger = keras.callbacks.CSVLogger('log.csv')
e_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', mode='min', patience=20)
cb_list = [csv_logger, e_stop]

history = model.fit_generator(generator = train_batch_generator,
                              #steps_per_epoch = (num_train_samples) // batch_size),
                              epochs = num_epochs,
                              verbose = 1,
                              validation_data = valid_batch_generator,
                              #validation_steps = (num_valid_samples) // batch_size),
                              callbacks = cb_list,
                              use_multiprocessing = True,
                              workers = 4,
                              max_queue_size = 10
                              )

## Evaluation

In [ ]:
# plot train history
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss (mae w angle wrapping)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['mae_wrap_angle'])
plt.plot(history.history['val_mae_wrap_angle'])
plt.title('Model loss metric mae w angle wrapping')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# evaluate model
test_batch_generator = DataGenerator_raw(partition['test'], **params)
score = model.evaluate_generator(test_batch_generator, verbose=1)
print('Test loss:', score[0])
print('Test mae w wrap:', score[1])
#print('Test mae w/o wrap:', score[2])

## Save

In [ ]:
# Save history to json file
import json
with open('../../../models_trained/m3_history.json', 'w') as f:
    json.dump(history.history, f)

In [ ]:
from keras.models import load_model
# Save model
model.save('../../../models_trained/m3_bs2048_adam_msaw_test-s18-s19-s20.h5')
del model